<center>
    <h1> ILI285 - Computación Científica I  / INF285 - Computación Científica </h1>
    <h2> Conjugate Gradient Method </h2>
    <h2> [[S]cientific [C]omputing [T]eam](#acknowledgements)</h2>
    <h2> Version: 1.11</h2>
</center>

## Table of Contents
* [Introduction](#intro)
* [Gradient Descent](#GDragon)
* [Conjugate Gradient Method](#CGM)
* [Conjugate Gradient Method with Preconditioning](#CGMp)
* [Let's Play: Practical Exercises and Profiling](#LP)
* [Acknowledgements](#acknowledgements)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_triangular
%matplotlib inline
# pip install memory_profiler
%load_ext memory_profiler

<div id='intro' />
## Introduction

Welcome to another edition of our IPython Notebooks. Here, we'll teach you how to solve $A\,x = b$ with $A$ being a _symmetric positive-definite matrix_, but the following methods have a key difference with the previous ones: these do not depend on a matrix factorization. The two methods that we'll see are called the Gradient Descent and the Conjugate Gradient Method. On the latter, we'll also see the benefits of preconditioning.

<div id='GDragon' />
## Gradient Descent

This is an iterative method. If you remember the iterative methods in the previous Notebook, to find the next approximate solution $\vec{x}_{k+1}$ you'd add a vector to the current approximate solution, $\vec{x}_k$, that is: $\vec{x}_{k+1} = \vec{x}_k + \text{vector}$. In this method, $\text{vector}$ is $\alpha_{k}\,\vec{r}_k$, where $\vec{r}_k$ is the residue ($\vec{b} - A\,\vec{x}_k$) and $\alpha_k = \cfrac{(\vec{r}_k)^T\,\vec{r}_k}{(\vec{r}_k)^T\,A\,\vec{r}_k}$, starting with some initial guess $\vec{x}_0$. Let's look at the implementation below:

In [ ]:
def gradient_descent(A, b, x0, n_iter=10):
    n = A.shape[0]
    #array with solutions
    X = np.empty((n_iter, n))
    X[0] = x0

    for k in range(1, n_iter):
        r = b - np.dot(A, X[k-1])
        if (all( v == 0 for v in r)): # The algorithm converged
            X[k:] = X[k-1]
            return X
            break
        alpha = np.dot(np.transpose(r), r)/np.dot(np.transpose(r), np.dot(A, r))
        X[k] = X[k-1] + alpha*r
    
    return X

Now let's try our algorithm! But first, let's borrow a function to generate a random symmetric positive-definite matrix, kindly provided by the previous notebook, and another one to calculate the vectorized euclidean metric.

In [ ]:
"""
Randomly generates an nxn symmetric positive-
definite matrix A.
"""
def generate_spd_matrix(n):
    A = np.random.random((n,n))
    #constructing symmetry
    A += A.T
    #symmetric+diagonally dominant -> symmetric positive-definite
    deltas = 0.1*np.random.random(n)
    row_sum = A.sum(axis=1)-np.diag(A)
    np.fill_diagonal(A, row_sum+deltas)
    return A

We'll try our algorithm with some matrices of different sizes, and we'll compare it with the solution given by Numpy's solver.

In [ ]:
A3 = generate_spd_matrix(3)
b3 = np.ones(3)
x30 = np.zeros(3)

X = gradient_descent(A3, b3, x30, 15)
sol = np.linalg.solve(A3, b3)
print (X[-1])
print (sol)
print (np.linalg.norm(X[-1] - sol)) # difference bewteen gradient_descent's solution and Numpy's solver's solution

In [ ]:
A10 = generate_spd_matrix(10)
b10 = np.ones(10)
x100 = np.zeros(10)

X = gradient_descent(A10, b10, x100, 15)
sol = np.linalg.solve(A10, b10)
print (X[-1])
print (sol)
print (np.linalg.norm(X[-1] - sol)) # difference bewteen gradient_descent's solution and Numpy's solver's solution

In [ ]:
A50 = generate_spd_matrix(50)
b50 = np.ones(50)
x500 = np.zeros(50)

X = gradient_descent(A50, b50, x500, 15)
sol = np.linalg.solve(A50, b50)
print (X[-1])
print (sol)
print (np.linalg.norm(X[-1] - sol)) # difference bewteen gradient_descent's solution and Numpy's solver's solution

As we can see, we're getting good solutions with 15 iterations, even for matrices on the bigger side. However, this method is not used too often; rather, its younger sibling, the Conjugate Gradient Method, is the more preferred choice.

<div id='CGM' />
## Conjugate Gradient Method

This method works by succesively eliminating the $n$ orthogonal components of the error, one by one. The method arrives at the solution with the following finite loop:

In [ ]:
def conjugate_gradient(A, b, x0):
    n = A.shape[0]
    X = np.empty((n, n))
    d = b - np.dot(A, x0)
    R = np.empty((n, n))
    X[0] = x0
    R[0] = b - np.dot(A, x0)

    for k in range(1, n):
        if (all( v == 0 for v in R[k-1])): # The algorithm converged
            X[k:] = X[k-1]
            return X
            break
        alpha = np.dot(np.transpose(R[k-1]), R[k-1]) / np.dot(np.transpose(d), np.dot(A, d))
        X[k] = X[k-1] + alpha*d
        R[k] = R[k-1] - alpha*np.dot(A, d)
        beta = np.dot(np.transpose(R[k]), R[k])/np.dot(np.transpose(R[k-1]), R[k-1])
        d = R[k] + beta*d

    return X

The science behind this algorithm is a bit long to explain, but for the curious ones, the explanation is on the official textbook (Numerical Analysis, 2nd Edition, Timothy Sauer). Now let's try it!

In [ ]:
A3 = generate_spd_matrix(3)
b3 = np.ones(3)
x30 = np.zeros(3)

X = conjugate_gradient(A3, b3, x30)
sol = np.linalg.solve(A3, b3)
print (X[-1])
print (sol)
print (np.linalg.norm(X[-1]- sol)) # difference bewteen conjugate_gradient's solution and Numpy's solver's solution

In [ ]:
A50 = generate_spd_matrix(50)
b50 = np.ones(50)
x500 = np.zeros(50)

X = conjugate_gradient(A50, b50, x500)
sol = np.linalg.solve(A50, b50)
print (X[-1])
print (sol)
print (np.linalg.norm(X[-1]-sol)) # difference bewteen conjugate_gradient's solution and Numpy's solver's solution

In [ ]:
A100 = generate_spd_matrix(100)
b100 = np.ones(100)
x1000 = np.zeros(100)

X = conjugate_gradient(A100, b100, x1000)
sol = np.linalg.solve(A100, b100)
print (X[-1])
print (sol)
print (np.linalg.norm(X[-1] - sol)) # difference bewteen conjugate_gradient's solution and Numpy's solver's solution

We can see that for small matrices the error for `gradient_descent` is somewhat smaller than the error for `conjugate_gradient`, but for big matrices this method has an extremely small error, practically zero. Isn't that amazing?!

Here are some questions for the student to think about:
* In which cases can the Conjugate Gradient Method converge in less than $n$ iterations?
* What will happen if you use the Gradient Descent or Conjugate Gradient Method with non-symmetric, non-positive-definite matrices?

<div id='CGMp' />
## Conjugate Gradient Method with Preconditioning

We've seen that the Conjugate Gradient Method works very well, but can we make it better? 

Very often, the convergence rate of iterative methods depends on the condition number of matrix $A$. By preconditioning, we'll reduce the condition number of the problem. The preconditioned version of the problem $A\,x = b$ is: $$M^{-1}\,A\,x = M^{-1}\,b$$ The matrix $M$ must be as close to $A$ as possible and easy to invert. One simple choice is the Jacobi Preconditioner $M = D$, since it shares its diagonal with $A$ and, as a diagonal matrix, is easy to invert. By applying this modification, we'll find that the method converges even faster.

In [ ]:
def diag_dot(D, v):
    n = len(D)
    sol = np.zeros(n)
    for i in range(n):
        sol[i] = D[i] * v[i]
    return sol

def conjugate_gradient_J(A, b, x0):
    M = np.diag(A)
    M_p = M**-1
    
    n = A.shape[0]
    X = np.empty((n, n))
    Z = np.empty((n, n))
    R = np.empty((n, n))
    X[0] = x0
    R[0] = b - np.dot(A, x0)
    Z[0] = diag_dot(M_p, R[0])
    d = diag_dot(M_p, R[0])

    for k in range(1, n):
        if (all( v == 0 for v in R[k-1]) and pr): # The algorithm converged
            X[k:] = X[k-1]
            return X
            break
        alpha = np.dot(np.transpose(R[k-1]), Z[k-1]) / np.dot(np.transpose(d), np.dot(A, d))
        X[k] = X[k-1] + alpha*d
        R[k] = R[k-1] - alpha*np.dot(A, d)
        Z[k] = diag_dot(M_p, R[k])
        beta = np.dot(np.transpose(R[k]), Z[k])/np.dot(np.transpose(R[k-1]), Z[k-1])
        d = Z[k] + beta*d

    return X

Now let's try it out:

In [ ]:
Aj100 = generate_spd_matrix(100)
bj100 = np.ones(100)
xj1000 = np.zeros(100)

X = conjugate_gradient_J(Aj100, bj100, xj1000)
X2 = conjugate_gradient(Aj100, bj100, xj1000)
sol = np.linalg.solve(Aj100, bj100)
print (np.linalg.norm(X[-1]- sol)) # difference bewteen gradient_descent's solution and Numpy's solver's solution
print (np.linalg.norm(X2[-1]- sol))

The absolute errors for both are very much alike and practically zero, but the difference is the _speed_ with which the error decreases, as we'll see in the Exercises section of this notebook.

Can you think of other preconditioners and try them out?

<div id='LP' />
## Let's Play: Practical Exercises and Profiling

First of all, define a function to calculate the progress of the relative error for a given method, that is, input the array of approximate solutions `X` and the real solution provided by Numpy's solver `r_sol` and return an array with the relative error for each step.

In [ ]:
def relative_error(X, r_sol):
    n_steps = X.shape[0]
    n_r_sol = np.linalg.norm(r_sol)
    E = np.zeros(n_steps)
    for i in range(n_steps):
        E[i] = np.linalg.norm(X[i] - r_sol) / n_r_sol
    return E

Try the three methods with a small non-symmetric, non-positive-definite matrix. Plot the relative error for all three methods.

In [ ]:
n = 10
B = 10 * np.random.random((n,n))
b = 10 * np.random.random(n)
x0 = np.zeros(n)

In [ ]:
X1 = gradient_descent(B, b, x0, n)
X2 = conjugate_gradient(B, b, x0)
X3 = conjugate_gradient_J(B, b, x0)
r_sol = np.linalg.solve(B, b)

E1 = relative_error(X1, r_sol)
E2 = relative_error(X2, r_sol)
E3 = relative_error(X3, r_sol)

In [ ]:
iterations = np.linspace(1, n, n)
plt.xlabel('Iteration')
plt.ylabel('Relative Error')
plt.title('Evolution of the Relative Error for each method')
plt.semilogy(iterations, E1, 'go', markersize=8) # Green spots are for Gradient Descent
plt.semilogy(iterations, E2, 'ro', markersize=8) # Red spots are for Conjugate Gradient
plt.semilogy(iterations, E3, 'co', markersize=8) # Cyan spots are for Conjugate Gradient with Jacobi Preconditioner
plt.show()

As you can see, if the matrix doesn't meet the requirements for these methods, the results can be quite terrible.

Let's try again, this time using an appropriate matrix.

In [ ]:
n = 100
A = 10 * generate_spd_matrix(n)
b = 10 * np.random.random(n)
x0 = np.random.random(n)

In [ ]:
X1 = gradient_descent(A, b, x0, n)
X2 = conjugate_gradient(A, b, x0)
X3 = conjugate_gradient_J(A, b, x0)
r_sol = np.linalg.solve(A, b)

E1 = relative_error(X1, r_sol)
E2 = relative_error(X2, r_sol)
E3 = relative_error(X3, r_sol)

In [ ]:
iterations = np.linspace(1, n, n)
plt.xlabel('Iteration')
plt.ylabel('Relative Error')
plt.title('Evolution of the Relative Error for each method')
plt.semilogy(iterations, E1, 'go', markersize=4) # Green spots are for Gradient Descent
plt.semilogy(iterations, E2, 'ro', markersize=4) # Red spots are for Conjugate Gradient
plt.semilogy(iterations, E3, 'co', markersize=4) # Cyan spots are for Conjugate Gradient with Jacobi Preconditioner
plt.grid(True)
plt.xlim(0,10)
plt.show()

Amazing! We started with a huge relative error and reduced it to practically zero in just under 6 iterations (the algorithms all have 100 iterations but we're showing you the first 10). We can clearly see that the Conjugate Gradient Method with Preconditioning needs the least of the three, with the Gradient Descent needing the most.

Let's try with an even bigger matrix!

In [ ]:
n = 1000
A = 10 * generate_spd_matrix(n)
b = 10 * np.random.random(n)
x0 = np.random.random(n)

In [ ]:
X1 = gradient_descent(A, b, x0, n)
X2 = conjugate_gradient(A, b, x0)
X3 = conjugate_gradient_J(A, b, x0)
r_sol = np.linalg.solve(A, b)

E1 = relative_error(X1, r_sol)
E2 = relative_error(X2, r_sol)
E3 = relative_error(X3, r_sol)

In [ ]:
iterations = np.linspace(1, n, n)
plt.xlabel('Iteration')
plt.ylabel('Relative Error')
plt.title('Evolution of the Relative Error for each method')
plt.semilogy(iterations, E1, 'go', markersize=4) # Green spots are for Gradient Descent
plt.semilogy(iterations, E2, 'ro', markersize=4) # Red spots are for Conjugate Gradient
plt.semilogy(iterations, E3, 'co', markersize=4) # Cyan spots are for Conjugate Gradient with Jacobi Preconditioner
plt.grid(True)
plt.xlim(0,10)
plt.show()

We can see that, reached a certain size for the matrix, the amount of iterations needed to reach a small error remains more or less the same. We encourage you to try other kinds of matrices to see how the algorithms behave, and experiment with the codes to your liking. Now let's move on to profiling.

Of course, you win some, you lose some. Accelerating the convergence of the algorithm means you have to spend more of other resources. We'll use the functions `%timeit` and `%memit` to see how the algorithms behave.

In [ ]:
A = generate_spd_matrix(100)
b = np.ones(100)
x0 = np.random.random(100)

In [ ]:
%timeit gradient_descent(A, b, x0, 100)
%timeit conjugate_gradient(A, b, x0)
%timeit conjugate_gradient_J(A, b, x0)

In [ ]:
# This does not seem to be working now, it needs to be updated.
#%memit gradient_descent(A, b, x0, 100)
#%memit conjugate_gradient(A, b, x0)
#%memit conjugate_gradient_J(A, b, x0)

We see something interesting here: all three algorithms need the same amount of memory.

What happened with the measure of time? Why is it so big for the algorithm that has the best convergence rate? Besides the end of the loop, we have one other criteria for stopping the algorithm: When the residue r reaches the _exact_ value of zero, we say that the algorithm converged, and stop. However it's very hard to get an error of zero for randomized initial guesses, so this almost never happens, and we can't take advantage of the convergence rate of the algorithms. 

There's a way we can fix this: instead of using this criteria, make the algorithm stop when a certain _tolerance_ is reached. That way, when the error gets small enough (which happens faster for the third method), we can stop and say that we got a good enough solution. We'll give the task of modifying the algorithms to let this happen.

You can try with different matrices, different initial conditions, different sizes, etcetera. Try some more plotting, profiling, and experimenting. Have fun!

<div id='acknowledgements' />
# Acknowledgements
* _Material created by professor Claudio Torres_ (`ctorres@inf.utfsm.cl`) _and assistants: Laura Bermeo, Alvaro Salinas, Axel Simonsen and Martín Villanueva. DI UTFSM. April 2016._